In [ ]:
! pip install ultralytics  # YOLOv9 may be part of the ultralytics package in the future, check for updates
! pip install opencv-python-headless
! pip install ipywebrtc


^C
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/bin/pip", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 68, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_internal/commands/__init__.py", line 109, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", l

: 

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the YOLOv8 model, specify the 'yolov8n.pt' weights or the closest available model
model = YOLO('yolov8n.pt')  # Adjust weight path as per your use case


HUMAN DETECTION

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load the YOLO model
model = YOLO('yolov8n.pt')  # Adjust this if you're using different weights

# Open the webcam (0 is typically the default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get the dimensions of the frame
    frame_height, frame_width = frame.shape[:2]
    camera_center_x, camera_center_y = frame_width // 2, frame_height // 2

    # Draw a yellow circle at the center of the camera view
    camera_center_color = (0, 255, 255)  # Yellow color
    cv2.circle(frame, (camera_center_x, camera_center_y), radius=20, color=camera_center_color, thickness=-1)

    # Run YOLO inference on the frame
    results = model(frame)
    
    # Process each detection and draw bounding boxes
    for result in results[0].boxes.data:
        # Extract bounding box coordinates and class ID
        x1, y1, x2, y2, confidence, class_id = result.cpu().numpy()
        
        # Check if the detected object is a person (typically class_id == 0 in COCO)
        if class_id == 0:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Define border color (red) and fill color (light green)
            border_color = (0, 0, 255)  # Red border
            fill_color = (144, 238, 144)  # Light green fill

            # Create an overlay to apply opacity
            overlay = frame.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), fill_color, -1)  # Filled rectangle
            cv2.addWeighted(overlay, 0.2, frame, 0.8, 0, frame)  # Blend with 30% opacity

            # Draw the rectangle border
            cv2.rectangle(frame, (x1, y1), (x2, y2), border_color, 2)

            # Draw the center point of the detected person as a purple circle
            person_center_x, person_center_y = (x1 + x2) // 2, (y1 + y2) // 2
            center_color = (128, 0, 128)  # Purple
            cv2.circle(frame, (person_center_x, person_center_y), radius=20, color=center_color, thickness=-1)

            # Draw an orange line between the camera center and the detected person's center
            line_color = (0, 165, 255)  # Orange
            cv2.line(frame, (camera_center_x, camera_center_y), (person_center_x, person_center_y), line_color, thickness=10)

    # Display the resulting frame
    cv2.imshow('YOLOv8 Real-Time Person Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()


2024-11-26 10:24:28.944 Python[7654:320348] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load the YOLO model
model = YOLO('yolov8n.pt')  # Adjust this if you're using different weights

# Open the webcam (0 is typically the default camera)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Placeholder for LIDAR distance, you can replace it with actual LIDAR readings
lidar_distance = 5  # Example distance from LIDAR in meters (adjust according to your setup)

# Known distance in pixels from the camera center to a reference object at the lidar_distance
# You would need to measure this for calibration.
reference_distance_pixels = 100  # Example distance in pixels, adjust it

# The scaling factor will convert pixels to meters
scaling_factor = lidar_distance / reference_distance_pixels

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get the dimensions of the frame
    frame_height, frame_width = frame.shape[:2]
    camera_center_x, camera_center_y = frame_width // 2, frame_height // 2

    # Draw a yellow circle at the center of the camera view
    camera_center_color = (0, 255, 255)  # Yellow color
    cv2.circle(frame, (camera_center_x, camera_center_y), radius=20, color=camera_center_color, thickness=-1)

    # Run YOLO inference on the frame
    results = model(frame)
    
    # Process each detection and draw bounding boxes
    for result in results[0].boxes.data:
        # Extract bounding box coordinates and class ID
        x1, y1, x2, y2, confidence, class_id = result.cpu().numpy()
        
        # Check if the detected object is a person (typically class_id == 0 in COCO)
        if class_id == 0:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Define border color (red) and fill color (light green)
            border_color = (0, 0, 255)  # Red border
            fill_color = (144, 238, 144)  # Light green fill

            # Create an overlay to apply opacity
            overlay = frame.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), fill_color, -1)  # Filled rectangle
            cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)  # Blend with 30% opacity

            # Draw the rectangle border
            cv2.rectangle(frame, (x1, y1), (x2, y2), border_color, 2)

            # Draw the center point of the detected person as a purple circle
            person_center_x, person_center_y = (x1 + x2) // 2, (y1 + y2) // 2
            center_color = (128, 0, 128)  # Purple
            cv2.circle(frame, (person_center_x, person_center_y), radius=20, color=center_color, thickness=-1)

            # Draw an orange line between the camera center and the detected person's center
            line_color = (0, 165, 255)  # Orange
            cv2.line(frame, (camera_center_x, camera_center_y), (person_center_x, person_center_y), line_color, thickness=10)

            # Calculate the pixel distance to the person
            distance_in_pixels = np.sqrt((person_center_x - camera_center_x) ** 2 + (person_center_y - camera_center_y) ** 2)

            # Calculate the distance in meters using the scaling factor
            distance_in_meters = distance_in_pixels * scaling_factor

            # Print the distance to the terminal
            print(f"Distance to person: {distance_in_meters:.2f} meters")

            # Display the distance on the video frame
            distance_text = f"Distance: {distance_in_meters:.2f} meters"
            cv2.putText(frame, distance_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('YOLOv8 Real-Time Person Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 83.1ms
Speed: 5.8ms preprocess, 83.1ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 13.01 meters

0: 384x640 2 persons, 2 chairs, 64.8ms
Speed: 12.0ms preprocess, 64.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 12.87 meters
Distance to person: 39.68 meters



2024-11-26 10:24:41.253 Python[7654:320348] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


0: 384x640 2 persons, 1 couch, 47.0ms
Speed: 3.5ms preprocess, 47.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 12.62 meters
Distance to person: 39.64 meters

0: 384x640 2 persons, 1 chair, 72.6ms
Speed: 1.4ms preprocess, 72.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 12.71 meters
Distance to person: 39.68 meters

0: 384x640 2 persons, 43.6ms
Speed: 1.6ms preprocess, 43.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 12.67 meters
Distance to person: 39.68 meters

0: 384x640 2 persons, 50.6ms
Speed: 10.3ms preprocess, 50.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 12.70 meters
Distance to person: 39.68 meters

0: 384x640 2 persons, 1 chair, 47.1ms
Speed: 1.4ms preprocess, 47.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 12.89 meters
Distance to person: 39.75 meters

0: 384x

: 

In [ ]:
import serial
import time

ser = serial.Serial('/dev/tty.usbmodem101', 115200, timeout=1)  # Replace '/dev/ttyUSB0' with your port

def read_distance():
    # Send read command
    command = bytes([0x01, 0x03, 0x00, 0x00, 0x00, 0x01, 0x84, 0x0A])
    ser.write(command)

    # Read response
    response = ser.read(9)

    if len(response) == 9 and response[0] == 0x01 and response[1] == 0x03:
        distance = (response[3] << 8) + response[4]
        return distance
    else:
        return None

if __name__ == "__main__":
    while True:
        distance = read_distance()
        if distance is not None:
            print("Distance:", distance, "cm")
        else:
            print("Failed to read distance")
        time.sleep(0.1)

In [ ]:
import serial
from collections import deque

# Initialize serial connection
ser = serial.Serial("/dev/tty.usbmodem101", baudrate=115200, timeout=0.1)

# Moving average buffer
distance_buffer = deque(maxlen=10)
strength_buffer = deque(maxlen=10)

try:
    while True:
        if ser.in_waiting > 0:
            byte = ser.read(1)
            if byte == b'\x59':  # First header byte
                second_byte = ser.read(1)
                if second_byte == b'\x59':  # Second header byte
                    data = ser.read(7)  # Remaining 7 bytes
                    if len(data) == 7:
                        frame = b'\x59\x59' + data
                        distance = int.from_bytes(frame[2:4], byteorder='little')
                        strength = int.from_bytes(frame[4:6], byteorder='little')
                        checksum = frame[8]

                        if sum(frame[:8]) & 0xFF == checksum:
                            distance_buffer.append(distance)
                            strength_buffer.append(strength)

                            average_distance = sum(distance_buffer) / len(distance_buffer)
                            average_strength = sum(strength_buffer) / len(strength_buffer)

                            print(f"Distance: {distance} cm, Strength: {strength}")
                            print(f"Smoothed Distance: {average_distance:.2f} cm, Strength: {average_strength:.2f}")
                        else:
                            print("Checksum failed!")
except KeyboardInterrupt:
    ser.close()


In [ ]:
! pip install pyautogui


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 KB 2.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 KB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.2/775.2 KB 3.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pyautogui
import time

# Message and repetition count
message = "as u say"
repetitions = 1000

# Instructions to the user
print("Open WhatsApp Desktop, click on the chat, and make sure the typing box is active.")
print("You have 10 seconds to switch to the app...")
time.sleep(10)

# Sending the message repeatedly
for _ in range(repetitions):
    pyautogui.typewrite(message+f"{_}")  # Type the message
    pyautogui.press("enter")  # Press Enter to send the message
    time.sleep(0.1)  # Optional: Small delay to avoid overwhelming the system


Open WhatsApp Desktop, click on the chat, and make sure the typing box is active.
You have 10 seconds to switch to the app...


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

def calculate_distance(left_x, right_x, focal_length, baseline):
    """
    Calculate distance to the object using stereo vision.
    :param left_x: x-coordinate of the object in the left camera.
    :param right_x: x-coordinate of the object in the right camera.
    :param focal_length: Focal length of the camera (in pixels).
    :param baseline: Distance between the two cameras (in meters).
    :return: Distance to the object (in meters).
    """
    disparity = left_x - right_x
    if disparity <= 0:
        return float('inf')  # No valid disparity
    return (focal_length * baseline) / disparity

# Initialize cameras and YOLO model
camera_left = cv2.VideoCapture(0)  # Camera 0
camera_right = cv2.VideoCapture(1)  # Camera 1
model = YOLO('yolov8n.pt')  # Load YOLOv8n model

# Camera parameters
focal_length = 600  # Approximate focal length in pixels (adjust for your camera)
baseline = 0.1  # Distance between the cameras in meters (10 cm as an example)

try:
    while True:
        # Capture frames from both cameras
        ret_left, frame_left = camera_left.read()
        ret_right, frame_right = camera_right.read()

        if not ret_left or not ret_right:
            print("Error capturing from one or both cameras")
            break

        # Detect objects in both frames using YOLO
        results_left = model(frame_left)
        results_right = model(frame_right)

        # Extract detections for persons
        persons_left = [
            box for box in results_left[0].boxes if box.cls[0] == 0
        ]  # Class ID 0 corresponds to 'person' in COCO
        persons_right = [
            box for box in results_right[0].boxes if box.cls[0] == 0
        ]

        # Match objects between cameras (simplistic matching by proximity in x-coordinates)
        for person_left in persons_left:
            # Extract the center x-coordinate from the left camera
            left_x, _, _, _ = person_left.xywh[0].tolist()

            closest_person = None
            min_diff = float('inf')

            for person_right in persons_right:
                # Extract the center x-coordinate from the right camera
                right_x, _, _, _ = person_right.xywh[0].tolist()

                diff = abs(left_x - right_x)
                if diff < min_diff:
                    min_diff = diff
                    closest_person = right_x

            # Calculate distance if a match is found
            if closest_person is not None:
                distance = calculate_distance(left_x, closest_person, focal_length, baseline)
                print(f"Distance to person: {distance:.2f} meters")

                # Draw bounding boxes and distance on the left frame
                x1, y1, x2, y2 = map(int, person_left.xyxy[0].tolist())  # Bounding box coordinates
                cv2.rectangle(frame_left, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_left, f"{distance:.2f}m", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display both frames
        cv2.imshow("Left Camera", frame_left)
        cv2.imshow("Right Camera", frame_right)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release cameras and close windows
    camera_left.release()
    camera_right.release()
    cv2.destroyAllWindows()


2024-11-25 15:18:50.794 Python[2001:28288] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



0: 384x640 2 persons, 1 chair, 65.4ms
Speed: 4.8ms preprocess, 65.4ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 37.9ms
Speed: 2.5ms preprocess, 37.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 0.13 meters
Distance to person: inf meters

0: 384x640 2 persons, 1 chair, 66.4ms
Speed: 1.8ms preprocess, 66.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 41.6ms
Speed: 1.8ms preprocess, 41.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


2024-11-25 15:18:53.546 Python[2001:28288] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Distance to person: inf meters
Distance to person: 0.12 meters

0: 384x640 2 persons, 1 chair, 39.4ms
Speed: 1.6ms preprocess, 39.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 42.2ms
Speed: 1.3ms preprocess, 42.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 0.11 meters
Distance to person: inf meters

0: 384x640 2 persons, 1 chair, 42.8ms
Speed: 1.6ms preprocess, 42.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 43.7ms
Speed: 2.0ms preprocess, 43.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Distance to person: 0.11 meters
Distance to person: inf meters

0: 384x640 2 persons, 1 chair, 57.7ms
Speed: 1.3ms preprocess, 57.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 47.4ms
Speed: 1.5ms preprocess, 47.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 64

KeyboardInterrupt: 